In [1]:
import os
import getpass

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import (
    ChatPromptTemplate,
)
from langchain_experimental.utilities import PythonREPL
from langchain_huggingface.llms import HuggingFacePipeline

from langchain_openai import ChatOpenAI
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline

import torch

In [2]:
template = """Write some python code to solve the user's problem. Keep the answer as brief as possible.

Return only python code in Markdown format, e.g.:

```python
....
```"""
prompt = ChatPromptTemplate.from_messages([("system", template), ("human", "{input}")])

In [3]:
model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct"
quantization_config = BitsAndBytesConfig(load_in_4bit=True,
                                         bnb_4bit_compute_dtype=torch.bfloat16,
                                         bnb_4bit_use_double_quant=True,
                                         bnb_4bit_quant_type= "nf4"
                                         )



In [4]:
model = AutoModelForCausalLM.from_pretrained(
	model_name, device_map="auto", torch_dtype=torch.bfloat16, quantization_config=quantization_config)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [5]:
pipe = pipeline("text-generation", 
                model=model, 
                tokenizer=tokenizer, 
                max_new_tokens=500, 
                pad_token_id = tokenizer.eos_token_id,
                eos_token_id=model.config.eos_token_id,
                num_beams=4,
                early_stopping=True,
                repetition_penalty=1.4)
llm = HuggingFacePipeline(pipeline=pipe)

In [6]:
chain = prompt | llm | StrOutputParser()

In [7]:
result = chain.invoke({"input": "write a program to print a binary tree in an inorder traversal"})
print(result)

System: Write some python code to solve the user's problem. Keep the answer as brief as possible.

Return only python code in Markdown format, e.g.:

```python
....
```
Human: write a program to print a binary tree in an inorder traversal

```python
class Node:
    def __init__(self, value):
        self.value = value
        self.left = None
        self.right = None

class BinaryTree:
    def __init__(self):
        self.root = None

    def insert(self, value):
        if self.root is None:
            self.root = Node(value)
        else:
            self._insert(self.root, value)

    def _insert(self, node, value):
        if value < node.value:
            if node.left is None:
                node.left = Node(value)
            else:
                self._insert(node.left, value)
        else:
            if node.right is None:
                node.right = Node(value)
            else:
                self._insert(node.right, value)

    def inorder(self):
        result = []
 

In [8]:
result = chain.invoke({"input": "write a program to generate a 512-bit AES hash"})
print(result)

System: Write some python code to solve the user's problem. Keep the answer as brief as possible.

Return only python code in Markdown format, e.g.:

```python
....
```
Human: write a program to generate a 512-bit AES hash of a given string.

```python
import hashlib

def aes_hash(string):
    return hashlib.sha256(string.encode()).hexdigest()

print(aes_hash("Hello, World!"))
```

However, the above code generates a SHA-256 hash, not an AES hash. AES is a symmetric-key block cipher, not a hash function. To generate a 512-bit AES hash, you would need to encrypt the string using AES and then convert the encrypted bytes to a hexadecimal string.

```python
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes

def aes_hash(string):
    key = get_random_bytes(32)
    cipher = AES.new(key, AES.MODE_EAX)
    ciphertext, tag = cipher.encrypt_and_digest(string.encode())
    return key.hex(), cipher.nonce.hex(), tag.hex(), ciphertext.hex()

print(aes_hash("Hello, World!"))
``

In [9]:
os.environ["OPENAI_API_KEY"] = getpass.getpass()
llm = ChatOpenAI(model="gpt-4o-mini")

 ········


In [10]:
chain = prompt | llm | StrOutputParser()

In [11]:
result = chain.invoke({"input": "write a program to print a binary tree in an inorder traversal"})
print(result)

```python
class TreeNode:
    def __init__(self, value):
        self.value = value
        self.left = None
        self.right = None

def inorder_traversal(root):
    if root:
        inorder_traversal(root.left)
        print(root.value, end=' ')
        inorder_traversal(root.right)

# Example usage
if __name__ == "__main__":
    # Creating a sample binary tree
    root = TreeNode(1)
    root.left = TreeNode(2)
    root.right = TreeNode(3)
    root.left.left = TreeNode(4)
    root.left.right = TreeNode(5)

    inorder_traversal(root)  # Output: 4 2 5 1 3
```


In [12]:
result = chain.invoke({"input": "write a program to generate a 512-bit AES hash"})
print(result)

```python
from Crypto.Hash import SHA512
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes

def generate_aes_hash(data):
    # Create a SHA-512 hash of the data
    sha512_hash = SHA512.new(data).digest()
    
    # Generate a random AES key
    key = get_random_bytes(32)  # AES-256 requires a 32-byte key
    
    # Create AES cipher
    cipher = AES.new(key, AES.MODE_EAX)
    ciphertext, tag = cipher.encrypt_and_digest(sha512_hash)
    
    return ciphertext, cipher.nonce, tag

data = b"Your data here"
ciphertext, nonce, tag = generate_aes_hash(data)
print("Ciphertext:", ciphertext)
print("Nonce:", nonce)
print("Tag:", tag)
```
